# 預測腳踏車每小時的租借量

## 資料準備

In [ ]:
global Path    
Path = "file:/home/spark/ntcu_workshop/"

In [ ]:
hour_df= spark.read.format('csv').option("header", 'true').load(Path+"data/hour.csv")
hour_df.count()

In [ ]:
hour_df=hour_df.drop("instant").drop("dteday").drop('yr').drop("casual").drop("registered")

In [ ]:
from pyspark.sql.functions import col  

In [ ]:
hour_df= hour_df.select([ col(column).cast("double").alias(column) 
                                          for column in hour_df.columns])

In [ ]:
hour_df.show(5)

In [ ]:
train_df, test_df = hour_df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

# 建立pipeline所需元件

## 這個範例前處理只需要VectorAssembler

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer,  VectorIndexer,VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
featuresCols = hour_df.columns[:-1]
print featuresCols 

In [ ]:
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="aFeatures")
vectorIndexer = VectorIndexer(inputCol="aFeatures", outputCol="features", maxCategories=24)

## 建立 Decision Tree

In [ ]:
dt = DecisionTreeRegressor(labelCol="cnt",featuresCol= 'features')
dt_pipeline = Pipeline(stages=[vectorAssembler,vectorIndexer ,dt])

## 使用pipeline進行資料處理

In [ ]:
dt_pipelineModel = dt_pipeline.fit(train_df)

In [ ]:
print dt_pipelineModel.stages[2].toDebugString[:500]

## 使用pipelineModel 進行預測

In [ ]:
predicted_df=dt_pipelineModel.transform(test_df)

In [ ]:
predicted_df.select('season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', \
                     'weathersit', 'temp', 'atemp', 'hum', 'windspeed','cnt','prediction').show(10)

# 評估模型的準確率

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
evaluator = RegressionEvaluator(labelCol='cnt',predictionCol='prediction',metricName="rmse")

In [ ]:
predicted_df=dt_pipelineModel.transform(test_df)
rmse = evaluator.evaluate(predicted_df)
rmse